In [53]:
#Importing libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import numpy as np


In [63]:
def compare_integers(a, b):
    # Determine the absolute values of the integers and the length of the longer integer's binary representation
    a_abs = abs(a)
    b_abs = abs(b)
    n = max(len(format(a_abs, 'b')), len(format(b_abs, 'b')))

    # Create our quantum circuit with the appropriate number of qubits ( I added two qubits os they act as our ancilla qubits for comparaison )
    qreg = QuantumRegister(n+2)
    creg = ClassicalRegister(n+2)
    qc = QuantumCircuit(qreg, creg)

    # This will initialize the qubits to the |0> state
    for i in range(n+2):
        qc.initialize([1,0], qreg[i])

    # Apply the controlled-X gates to compare the numbers
    for i in range(n):
        if i < len(format(a_abs, 'b')):
            if format(a_abs, 'b')[-(i+1)] == '1':
                qc.cx(qreg[i], qreg[n])
        else:
            qc.cx(qreg[n+1], qreg[i])

        if i < len(format(b_abs, 'b')):
            if format(b_abs, 'b')[-(i+1)] == '1':
                qc.cx(qreg[i], qreg[n])
        else:
            qc.cx(qreg[n+1], qreg[i])

    # Hadamard gates and controlled-shift will implement QPE algorithm to determine the relative phase between the binary integers of a & b
    for i in range(n):
        qc.h(i)
        qc.cp(np.pi/2**(n-i), qreg[i], qreg[n])
        qc.h(i)

    # The Toffoli gate applied to the two most significant qubits of the input registers, with the target qubit being an additional ancilla qubit initialized to |1⟩ 
    # will determine the relative phase between a & b
    qc.ccx(qreg[n-1], qreg[n], qreg[n+1])

    # Finally, the Hadamard gate will encode the relative phase into an overall phase
    qc.h(n+1)

    # Measure the additional ancilla qubit
    qc.measure(qreg[n+1], creg[n+1])

    # Get the measurement outcome
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1)
    result = job.result().get_counts(qc)
    sign = -1 if '1' in result else 1

    # Determine the larger number and return it with the determined sign
    if a*sign > b*sign:
        print(qc)

        return a
        
        
    else:
        print(qc)
        return b
        
    


In [64]:
# Comparing two negative integers
compare_integers(-5000,-3)


         ┌─────────────────┐     ┌───┐                                        »
 q287_0: ┤ Initialize(1,0) ├──■──┤ H ├────────────────────────────────────────»
         ├─────────────────┤  │  └───┘     ┌───┐                              »
 q287_1: ┤ Initialize(1,0) ├──┼─────────■──┤ H ├──────────────────────────────»
         ├─────────────────┤  │  ┌───┐  │  ├───┤                              »
 q287_2: ┤ Initialize(1,0) ├──┼──┤ X ├──┼──┤ H ├──────────────────────────────»
         ├─────────────────┤  │  └─┬─┘  │  └───┘┌───┐┌───┐                    »
 q287_3: ┤ Initialize(1,0) ├──┼────┼────┼────■──┤ X ├┤ H ├────────────────────»
         ├─────────────────┤  │    │    │    │  └─┬─┘└───┘┌───┐┌───┐          »
 q287_4: ┤ Initialize(1,0) ├──┼────┼────┼────┼────┼───────┤ X ├┤ H ├──────────»
         ├─────────────────┤  │    │    │    │    │       └─┬─┘└───┘┌───┐┌───┐»
 q287_5: ┤ Initialize(1,0) ├──┼────┼────┼────┼────┼─────────┼───────┤ X ├┤ H ├»
         ├─────────────────┤  │    │    

-3

In [51]:
#Comparing two positive integers
compare_integers(34,90)

        ┌─────────────────┐┌───┐                                         ┌───┐»
q252_0: ┤ Initialize(1,0) ├┤ H ├───────────────────────────────■─────────┤ H ├»
        ├─────────────────┤└───┘          ┌───┐                │         └───┘»
q252_1: ┤ Initialize(1,0) ├──■─────────■──┤ H ├────────────────┼──────────────»
        ├─────────────────┤  │  ┌───┐  │  └───┘                │              »
q252_2: ┤ Initialize(1,0) ├──┼──┤ H ├──┼───────────────────────┼──────────────»
        ├─────────────────┤  │  └───┘  │       ┌───┐           │              »
q252_3: ┤ Initialize(1,0) ├──┼─────────┼────■──┤ H ├───────────┼──────────────»
        ├─────────────────┤  │         │    │  └───┘┌───┐      │              »
q252_4: ┤ Initialize(1,0) ├──┼─────────┼────┼────■──┤ H ├──────┼──────────────»
        ├─────────────────┤  │         │    │    │  └───┘┌───┐ │              »
q252_5: ┤ Initialize(1,0) ├──┼─────────┼────┼────┼────■──┤ H ├─┼──────────────»
        ├─────────────────┤  │  ┌───┐  │

90

In [52]:
#Comparing one positive integer and one negative
compare_integers(500,-37)

         ┌─────────────────┐     ┌───┐                                        »
 q259_0: ┤ Initialize(1,0) ├──■──┤ H ├────────────────────────────────────────»
         ├─────────────────┤  │  ├───┤                                        »
 q259_1: ┤ Initialize(1,0) ├──┼──┤ H ├────────────────────────────────────────»
         ├─────────────────┤  │  └───┘          ┌───┐                         »
 q259_2: ┤ Initialize(1,0) ├──┼─────────■────■──┤ H ├─────────────────────────»
         ├─────────────────┤  │  ┌───┐  │    │  └───┘                         »
 q259_3: ┤ Initialize(1,0) ├──┼──┤ H ├──┼────┼────────────────────────────────»
         ├─────────────────┤  │  └───┘  │    │       ┌───┐                    »
 q259_4: ┤ Initialize(1,0) ├──┼─────────┼────┼────■──┤ H ├────────────────────»
         ├─────────────────┤  │         │    │    │  └───┘     ┌───┐          »
 q259_5: ┤ Initialize(1,0) ├──┼─────────┼────┼────┼────■────■──┤ H ├──────────»
         ├─────────────────┤  │         

500

In [65]:
#comparing one positive integer and one negative
compare_integers(90,-7)

        ┌─────────────────┐     ┌───┐                                        »
q294_0: ┤ Initialize(1,0) ├──■──┤ H ├────────────────────────────────────────»
        ├─────────────────┤  │  └───┘     ┌───┐                              »
q294_1: ┤ Initialize(1,0) ├──┼────■────■──┤ H ├──────────────────────────────»
        ├─────────────────┤  │    │    │  └───┘┌───┐                         »
q294_2: ┤ Initialize(1,0) ├──┼────┼────┼────■──┤ H ├─────────────────────────»
        ├─────────────────┤  │    │    │    │  └───┘┌───┐┌───┐               »
q294_3: ┤ Initialize(1,0) ├──┼────┼────┼────┼────■──┤ X ├┤ H ├───────────────»
        ├─────────────────┤  │    │    │    │    │  └─┬─┘└───┘┌───┐┌───┐     »
q294_4: ┤ Initialize(1,0) ├──┼────┼────┼────┼────┼────┼────■──┤ X ├┤ H ├─────»
        ├─────────────────┤  │    │    │    │    │    │    │  └─┬─┘└───┘┌───┐»
q294_5: ┤ Initialize(1,0) ├──┼────┼────┼────┼────┼────┼────┼────┼───────┤ X ├»
        ├─────────────────┤  │    │    │    │    │  

90